In [ ]:
import scipy.io
import numpy as np
# from sklearn.preprocessing import StandardScaler


In [ ]:
data=scipy.io.loadmat('tract_analysis.mat')
data_all=data['data_all']

In [ ]:
def cost_lda(X,y):
    #ldr value
    #X is data, y is labeling
    nfeature=X.shape[1]

    ind0=np.where(y==0)
    ind1=np.where(y==1)
    ind2=np.where(y==2)
   
    X_0=np.reshape(np.squeeze(X[ind0,:]),[-1,nfeature])
    X_1=np.reshape(np.squeeze(X[ind1,:]),[-1,nfeature])
    X_2=np.reshape(np.squeeze(X[ind2,:]),[-1,nfeature])

    m0=np.mean(X_0,axis=0)
    m1=np.mean(X_1,axis=0)
    m2=np.mean(X_2,axis=0)
    mm=np.mean(X,axis=0)

    X_0_dm=X_0-m0
    X_1_dm=X_1-m1
    X_2_dm=X_2-m2
    X_dm=X-mm

    m0m=np.reshape(m0-mm,[-1,1])
    m1m=np.reshape(m1-mm,[-1,1])
    m2m=np.reshape(m2-mm,[-1,1])

    sw=np.matmul(np.transpose(X_0_dm),X_0_dm)+np.matmul(np.transpose(X_1_dm),X_1_dm)+np.matmul(np.transpose(X_2_dm),X_2_dm)
    sb=5*np.matmul(m0m,np.transpose(m0m))+3*np.matmul(m1m,np.transpose(m1m))+3*np.matmul(m2m,np.transpose(m2m))

    cost1=np.linalg.det(sb)/np.linalg.det(sw)
    return cost1

In [ ]:
#LDA
# run LDA, calculate the ldr

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf1 = LinearDiscriminantAnalysis(n_components=1,solver='eigen',shrinkage='auto')

# scaler=StandardScaler()

data_LDA_1d=np.zeros((8,11,1,3))
cost_LDA_unpermu_1d=np.zeros((8,3))

labels_true=np.array([0,0,0,0,0,1,1,1,2,2,2])
labels_test=labels_true

for isubj in range(8):
    for itest in range(3):

        if itest == 0:
            datasubj=data_all[isubj,:,:]
        elif itest == 1:
            datasubj=data_all[isubj,:,:3]
        elif itest == 2:
            datasubj=data_all[isubj,:,3:]
        datasubj_7f=datasubj
        clf1.fit(datasubj_7f, labels_true)
        data_LDA_1d[isubj,:,:,itest]=clf1.transform(datasubj_7f)
        
        cost_LDA_unpermu_1d[isubj,itest]=cost_lda(data_LDA_1d[isubj,:,:,itest],labels_test)

# print(cost_LDA_unpermu_1d)





In [ ]:
# permutation test

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf1 = LinearDiscriminantAnalysis(n_components=1,solver='eigen',shrinkage='auto')

# scaler=StandardScaler()

data_LDA_1d=np.zeros((8,11,1))

print(data_LDA_1d.shape)

nperm=100000;

cost_LDA_1d=np.zeros((8,3,nperm))

labels_true=np.array([0,0,0,0,0,1,1,1,2,2,2])

for iperm in range(nperm):
    labels_test=np.random.permutation(labels_true)
#     labels_test=labels_true
    if iperm%10000 == 0:
        print(iperm)
        
    for isubj in range(8):
        for itest in range(3):

            if itest == 0:
                datasubj=data_all[isubj,:,:]
            elif itest == 1:
                datasubj=data_all[isubj,:,:3]
            elif itest == 2:
                datasubj=data_all[isubj,:,3:]

            datasubj_7f=datasubj
            clf1.fit(datasubj_7f, labels_test)
            data_LDA_1d[isubj,:,:]=clf1.transform(datasubj_7f)
            cost_LDA_1d[isubj,itest,iperm]=cost_lda(data_LDA_1d[isubj,:,:],labels_test)
        
print('finished')





In [ ]:
# scipy.io.savemat('run100000_shrink_python.mat',{'cost_LDA_1d': cost_LDA_1d})
datetest=scipy.io.loadmat('run100000_shrink_python.mat')
cost_LDA_1d=datetest['cost_LDA_1d']

In [ ]:
#calculate p value 
#validated, it is right

def calc_pval(X,xx):
    n1=np.prod(X.shape)
    nn2=X[X>xx]
    n2=np.prod(nn2.shape)
    rr=(n2+1)/(n1+1)
    return rr

In [ ]:

p_all1d=np.zeros((8,3))

for isubj in range(8):
    for itest in range(3):
        p_all1d[isubj,itest]=calc_pval(cost_LDA_1d[isubj,itest,:],cost_LDA_unpermu_1d[isubj,itest])

# print(p_all1d)
